从海量语料资源中找寻信息需要依赖于信息检索的方法，信息检索无论是谷歌还是百度都离不开TF-IDF算法，但TF-IDF是万能的吗？并不是，它简单有效但缺乏语义特征，本节介绍比TF-IDF还牛逼的含有语义特征的信息检索方法 

## 1. TF-IDF模型
TF(term  frequency)，表示一个词在一个文档中出现的频率；IDF(inverse document  frequency)，表示一个词出现在多少个文档中。  

它的思路是这样的：同一个词在短文档中出现的次数和在长文档中出现的次数一样多时，对于短文档价值更大；一个出现概率很低的词一旦出现在文档中，其价值应该大于其他普遍出现的词。  

这在信息检索领域的向量模型中做相似度计算非常有效，屡试不爽，曾经是google老大哥发家的必杀技。但是在开发聊天机器人这个事情上看到了它的软肋，那就是它只是考虑独立的词上的事情，并没有任何语义信息在里面，因此我们需要选择加入了语义特征的更有效的信息检索模型。  

## 2. 隐含语义索引模型
在TF-IDF模型中，所有词构成一个高维的语义空间，每个文档在这个空间中被映射为一个点，这种方法维数一般比较高而且每个词作为一维割裂了词与词之间的关系。所以为了解决这个问题，我们要把词和文档同等对待，构造一个维数不高的语义空间，每个词和每个文档都是被映射到这个空间中的一个点。用数学来表示这个思想就是说，我们考察的概率即包括文档的概率，也包括词的概率，以及他们的联合概率。  

为了加入语义方面的信息，我们设计一个假想的隐含类包括在文档和词之间，具体思路是这样的：  

* （1）$选择一个文档的概率是p(d)$;  

* （2）$找到一个隐含类的概率是p(z|d)$;  

* （3）$生成一个词w的概率为p(w|z)$;  

$以上是假设的条件概率，我们根据观测数据能估计出来的是p(d, w)联合概率，这里面的z是一个隐含变量，表达的是一种语义特征。那么我们要做的就是利用p(d, w)来估计p(d)、p(z|d)和p(w|z)，最终根据p(d)、p(z|d)和p(w|z)来求得更精确的p(w, d)，即词与文档之间的相关度。$  

为了做更精确的估计，设计优化的目标函数是对数似然函数：  
$$L=\sum\sum{n(d,w)log(P(d, w))}$$
通过机器学习来训练这些概率，首先我们知道：
$$P(d,w)=P(d)\times{P(w|d)}$$
而
$$P(w|d)=\sum{P(w|z)P(z|d)}$$
同时又有：
$$P(z|d)=\frac{P(z)P(d|z)}{\sum{P(z)P(d|z)}}$$
那么：
$$P(d,w)=P(d)\times{\frac{\sum{P(w|z)P(z)P(d|z)}}{\sum{P(z)P(d|z)}}}=\sum{P(z)}\times{P(w|z)}\times{P(d|z)}$$
<br/>  
下面我们采取EM算法，EM算法的精髓就是按照最大似然的原理，先随便拍一个分布参数，让每个人都根据分布归类到某一部分，然后根据这些归类来重新统计数目，按照最大似然估计分布参数，然后再重新归类、调参、估计、归类、调参、估计，最终得出最优解  

那么我们要把每一个训练数据做归类，即p(z|d,w)，那么这个概率值怎么计算呢？  

我们先拍一个p(z)、p(d|z)、p(w|z)  

然后根据:  
$$P(z|d, w)=\frac{P(z)P(d|z)P(w|z)}{\sum{P(z)P(d|z)P(w|z)}}$$
其中分子是一个z，分母是所有的z的和
这样计算出来的值是p(z|d,w)的最大似然估计的概率估计（这是E过程）  

然后根据这个估计来对每一个训练样本做归类  

根据归类好的数据统计出$n(d,w)$  

